In [ ]:
!pip install yfinance

In [ ]:
import warnings
import pandas as pd
import numpy as np
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

import plotly.express as px

import yfinance as yf

In [ ]:
start_date = "2017-01-01"
end_date = "2023-01-01"
symbol = "BTC-USD"
data = yf.download(symbol, start=start_date, end=end_date)
data = data[["Open", "High", "Low", "Close", "Volume"]]
df = data.copy()
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000


In [ ]:
df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2191 entries, 2017-01-01 to 2022-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2191 non-null   float64
 1   High    2191 non-null   float64
 2   Low     2191 non-null   float64
 3   Close   2191 non-null   float64
 4   Volume  2191 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 102.7 KB


In [ ]:
df.describe()

,Open,High,Low,Close,Volume
count,2191.000000,2191.000000,2191.000000,2191.000000,2.191000e+03
mean,17612.752266,18058.205747,17113.597100,17617.805157,2.256631e+10
std,16747.708450,17176.684508,16246.748388,16739.789974,2.028196e+10
min,775.177979,823.307007,755.755981,777.757019,6.085170e+07
25%,6418.099854,6536.165039,6318.800049,6417.987549,5.303753e+09
50%,9664.904297,9834.716797,9460.571289,9665.533203,1.974523e+10
75%,23692.820312,24162.116211,23078.830078,23696.078125,3.317659e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,3.509679e+11




#Prepare Data

In [ ]:
def clean_data(df):
    return pd.DataFrame({'Date': df.index, 'Close': df['Close'], 'Volume': df['Volume']})

In [ ]:
df = clean_data(df)
df.head()

,Date,Close,Volume
Date,,,
2017-01-01,2017-01-01,998.325012,147775008
2017-01-02,2017-01-02,1021.750000,222184992
2017-01-03,2017-01-03,1043.839966,185168000
2017-01-04,2017-01-04,1154.729980,344945984
2017-01-05,2017-01-05,1013.380005,510199008


In [ ]:
df['Target'] = df[['Close']].shift(-1)
df.head()

,Date,Close,Volume,Target
Date,,,,
2017-01-01,2017-01-01,998.325012,147775008,1021.750000
2017-01-02,2017-01-02,1021.750000,222184992,1043.839966
2017-01-03,2017-01-03,1043.839966,185168000,1154.729980
2017-01-04,2017-01-04,1154.729980,344945984,1013.380005
2017-01-05,2017-01-05,1013.380005,510199008,902.200989


In [ ]:
# Remove the last row as it will be a null value
df = df[:-1]
df.head()

,Date,Close,Volume,Target
Date,,,,
2017-01-01,2017-01-01,998.325012,147775008,1021.750000
2017-01-02,2017-01-02,1021.750000,222184992,1043.839966
2017-01-03,2017-01-03,1043.839966,185168000,1154.729980
2017-01-04,2017-01-04,1154.729980,344945984,1013.380005
2017-01-05,2017-01-05,1013.380005,510199008,902.200989


#Test Train Split

In [ ]:
df_tts = df.copy()
df_tts = df_tts.drop(columns = ['Date'])

In [ ]:
X = df_tts.iloc[:, : -1]
y = df_tts.iloc[:, -1]
X.shape, y.shape

((2190, 2), (2190,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)

Shape of X_train:  (1752, 2)
Shape of y_train:  (1752,)


#Ridge Regression

In [ ]:
ridge_model = Ridge()
ridge_model.fit(X_train, y_train)

Ridge()

In [ ]:
y_pred = ridge_model.predict(X_test)
ridge_error = np.sqrt(mean_squared_error(y_test, y_pred))
print("mean_squared_error: ", ridge_error)

mean_squared_error:  904.5385939073617


In [ ]:
# Make Prediction
predicted_prices = ridge_model.predict(X)
predicted_prices

array([ 1025.0041899 ,  1048.45746767,  1070.45373415, ...,
       16561.47979201, 16648.19223293, 16610.16640818])

In [ ]:
# Append the predicted values into a list
Predicted = []
for i in predicted_prices:
  Predicted.append(i)
len(Predicted)

2190

In [ ]:
# Append the close values to the list
close = []
for i in df["Close"]:
  close.append(i)
len(close)

2190

In [ ]:
# Create a dataframe based on the dates
df_predicted = df[['Date']]
df_predicted.head()

,Date
Date,
2017-01-01,2017-01-01
2017-01-02,2017-01-02
2017-01-03,2017-01-03
2017-01-04,2017-01-04
2017-01-05,2017-01-05


In [ ]:
# Add the close values to the dataframe
df_predicted['Close'] = close
df_predicted.head()

,Date,Close
Date,,
2017-01-01,2017-01-01,998.325012
2017-01-02,2017-01-02,1021.750000
2017-01-03,2017-01-03,1043.839966
2017-01-04,2017-01-04,1154.729980
2017-01-05,2017-01-05,1013.380005


In [ ]:
# Add the predicted values to the dataframe
df_predicted['Prediction'] = Predicted
df_predicted.head()

,Date,Close,Prediction
Date,,,
2017-01-01,2017-01-01,998.325012,1025.004190
2017-01-02,2017-01-02,1021.750000,1048.457468
2017-01-03,2017-01-03,1043.839966,1070.453734
2017-01-04,2017-01-04,1154.729980,1181.261517
2017-01-05,2017-01-05,1013.380005,1040.430682


In [ ]:
# Function to plot interactive plots using Plotly Express
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
# Plot the results
interactive_plot(df_predicted, "Original vs Prediction")

#XGBoost

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = xgb_model.predict(X_test)
xgb_error = np.sqrt(mean_squared_error(y_test, y_pred))
print("mean_squared_error: ", xgb_error)

mean_squared_error:  1221.1655406932778


In [ ]:
# Make Prediction
predicted_prices = xgb_model.predict(X)
predicted_prices

array([ 1012.5506,  1040.4333,  1045.1437, ..., 16582.217 , 16911.982 ,
       16363.036 ], dtype=float32)

In [ ]:
# Append the predicted values into a list
Predicted = []
for i in predicted_prices:
  Predicted.append(i)
len(Predicted)

2190

In [ ]:
# Append the close values to the list
close = []
for i in df["Close"]:
  close.append(i)
len(close)

2190

In [ ]:
# Create a dataframe
df_predicted = df[['Date']]
df_predicted['Close'] = close
df_predicted['Prediction'] = Predicted
df_predicted.head()

,Date,Close,Prediction
Date,,,
2017-01-01,2017-01-01,998.325012,1012.550598
2017-01-02,2017-01-02,1021.750000,1040.433350
2017-01-03,2017-01-03,1043.839966,1045.143677
2017-01-04,2017-01-04,1154.729980,1115.032715
2017-01-05,2017-01-05,1013.380005,1016.309875


In [ ]:
# Plot the results
interactive_plot(df_predicted, "Original vs Prediction")